# Prepare Dataset
- Read raw session files that are annoted (include .json file)
- decimate sessions files from 100 Hz to 20 Hz
- Window the data into 100 datapoint (in 5 Hz) long windows of xyz
- Save xyz acc data and the labels to pytorch dataset files in:
    - `pipeline/datasets/train_dataset` 
    - `pipeline/datasets/test_dataset` with specified `test_size` 
    - `pipeline/datasets/<session_name>-holdout_dataset` with specified `n_sessions_holdout`

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
import seaborn as sns
import plotly.express as px
import json
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
from torch.utils.data import TensorDataset,DataLoader
from tqdm import tqdm

# Process Raw Data

In [ ]:
# make directories
if not os.path.isdir('pipeline'):
    os.system('mkdir -p pipeline')
else:
    print("pipeline directory already exists - delete or rename it")

In [ ]:
json_labels = {}

total_length = 0
nfiles = 5     # set to limit number of sessions to read
i=0

raw_dir = './data'

os.system('mkdir pipeline/1_dm')

for subdir in os.listdir(raw_dir):
    if os.path.exists(f'{raw_dir}/{subdir}/{subdir}_data.json'):
        # json files with annotations exists - dataset is annotated
        i += 1

        # Read labels
        with open(f'{raw_dir}/{subdir}/{subdir}_data.json', 'r') as f:
            json_labels[subdir] = json.load(f)
        
        # if dataset is annotated, read raw data file, decimate from 100 to 20 Hz, and save just x,y,z in 1_dm dir
        df = pd.read_csv(f'{raw_dir}/{subdir}/raw_data.csv', header=None, usecols=[2,3,4], names=['x','y','z'])
        json_labels[subdir]['length'] = len(df[::5])
        total_length += len(df[::5])
        df[::5].to_csv(f'pipeline/1_dm/{subdir}.csv', index=False)

        print(f'{subdir} - Length: {len(df[::5])}')

    else:
        print(f'Skipped {subdir}')
        
    if i == nfiles:
        break

In [ ]:
# Window data

os.system('mkdir pipeline/2_windowed')

for i,file in enumerate(os.listdir('pipeline/1_dm/')):
    df = pd.read_csv(f'pipeline/1_dm/{file}')
    print(f'{file} - Before {df.shape}')
    w = np.empty((len(df)-99, 300), dtype=float)
    for j in range(len(df)-99):
        w[j] = df[j:j+100].to_numpy().T.flatten()
    print(f'After {w.shape}')
    np.save(f'pipeline/2_windowed/{file.split(".")[0]}.npy', w)

In [ ]:
# Make Labels from annotations

os.system('mkdir pipeline/labels')

for i, (key, annot) in enumerate(json_labels.items()):
    l = np.zeros(annot['length'] - 99)
    for j in range(annot['start']//5, annot['end']//5):
        for puff in annot['puffs']:
            if j >= puff['start']//5 and j <= puff['end']//5:
                l[j] = 1
    
    np.save(f'pipeline/labels/{key}.npy', l)

In [ ]:
# # visualize true labels on continous signal
# i = 21
# labels = np.load(f'pipeline/labels/{i}.npy')

# df = pd.read_csv(f'pipeline/1_dm/{i}.csv')
# df['label'] = np.pad(labels*10, (0,99), mode='constant', constant_values=0)

# fig = px.line(data_frame=df)
# fig.show(renderer='browser')

In [ ]:
# Prepare Pytorch Datasets
n_sessions_holdout = 2
test_size = 0.3

os.system('mkdir pipeline/datasets')

X = np.zeros((0,300))
y = np.zeros((0,1))

for i,file in enumerate(os.listdir('pipeline/2_windowed')):
    # First, save <n_sessions_holdout> sessions
    if i < n_sessions_holdout:
        session_name = file.split('.')[0]
        X_holdout = torch.from_numpy(np.load(f'pipeline/2_windowed/{file}')).float()
        y_holdout = torch.from_numpy(np.load(f'pipeline/labels/{file}').reshape(-1,1)).float()
        torch.save(TensorDataset(X_holdout, y_holdout), f'pipeline/datasets/{session_name}-holdout.pt')
        i += 1

    X = np.concatenate([X, np.load(f'pipeline/2_windowed/{file}')])
    y = np.concatenate([y, np.load(f'pipeline/labels/{file}').reshape(-1,1)])

(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=test_size, stratify=y, shuffle=True, random_state=0)

train_dataset = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
test_dataset = TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())

torch.save(train_dataset, 'pipeline/datasets/train_dataset.pt')
torch.save(test_dataset, 'pipeline/datasets/test_dataset.pt')